# Fix district name discrepencies between IDSR and geographic boundary files

There were many districts in the IDSR dataset that did not match the districts by name in the geographic boundaries file. These either came in the form of:
1. Names that were originally in a language other than English that were then spelled slightly different when transliterated to English characters.
2. Names that were simply different either because it was changed or one of the datasets had incorrect information.

To resolve the first one I:
1. Used cosine similarity with country, province, and district name to match those with a similar enough name.
2. Used the phonetic spelling to match those spelled different but pronounced similar enough.

To resolve the second, I manually researched on the Web the remaining districts not matched by the above code and was able to find the matching district.

In [ ]:
import pandas as pd
import numpy as np
from sqlalchemy import create_engine

db_connection_url = "postgresql://<dbuser>:<dbpasswd>@<dbhost>:<dbport>/<dbname>"
con = create_engine(db_connection_url, isolation_level="AUTOCOMMIT")

In [ ]:
def soundex(name, len=4):
    """ soundex module conforming to Odell-Russell algorithm """

    # digits holds the soundex values for the alphabet
    soundex_digits = '01230120022455012623010202'
    sndx = ''
    fc = ''

    # Translate letters in name to soundex digits
    for c in name.upper(  ):
        if c.isalpha(  ):
            if not fc: fc = c   # Remember first letter
            d = soundex_digits[ord(c)-ord('A')]
            # Duplicate consecutive soundex digits are skipped
            if not sndx or (d != sndx[-1]):
                sndx += d

    # Replace first digit with first letter
    sndx = fc + sndx[1:]

    # Remove all 0s from the soundex code
    sndx = sndx.replace('0', '')

    # Return soundex code truncated or 0-padded to len characters
    return (sndx + (len * '0'))[:len]

In [ ]:
import phonetics
import Levenshtein as lev

# -- weight
weight = {
    phonetics.soundex: 0.2,
    phonetics.dmetaphone: 0.2,
    phonetics.metaphone: 0.5,
    phonetics.nysiis: 0.1
}

# -- algorithms
algorithms = [phonetics.soundex, phonetics.dmetaphone, phonetics.metaphone, phonetics.nysiis]
    

num2words = {1: 'One', 2: 'Two', 3: 'Three', 4: 'Four', 5: 'Five', \
             6: 'Six', 7: 'Seven', 8: 'Eight', 9: 'Nine'}
def has_numbers(inputString):
    return any(char.isdigit() for char in inputString)


def compare_words_similarity(word1, word2):
    # -- total
    word1 = word1.replace("'", "").replace('(', '').replace(')', '').replace('.', ' ').replace('/', ' ').replace('-', ' ').replace('&', ' ')
    word2 = word2.replace("'", "").replace('(', '').replace(')', '').replace('.', ' ').replace('/', ' ').replace('-', ' ').replace('&', ' ')
    if has_numbers(word1):
        word1 = ''.join([' ' + num2words[int(char)] if char.isdigit() else char for char in word1])
    if has_numbers(word2):
        word2 = ''.join([' ' + num2words[int(char)] if char.isdigit() else char for char in word2])
    total = 0.0
    for entry in algorithms:
        if entry is phonetics.soundex:
            c = []
            for w in word1.split(' '):
                if len(w.strip()) > 0:
                    s_len = len(w)-1 if len(w) < 6 else 6
                    c.append(entry(w, s_len))
            if len(c) == 0:
                continue
            code1 = ' '.join(c)
            c = []
            for w in word2.split(' '):
                if len(w.strip()) > 0:
                    s_len = len(w)-1 if len(w) < 6 else 6
                    try:
                        c.append(entry(w, s_len))
                    except IndexError as ex:
                        continue
            if len(c) == 0:
                continue
            code2 = ' '.join(c)
        else:
            c = []
            for w in word1.split(' '):
                if len(w.strip()) > 0:
                    try:
                        c.append(''.join(entry(w)))
                    except IndexError as ex:
                        continue
            if len(c) == 0:
                continue
            code1 = ' '.join(c)
            c = []
            for w in word2.split(' '):
                if len(w.strip()) > 0:
                    try:
                        c.append(''.join(entry(w.strip())))
                    except IndexError as ex:
                        continue
            if len(c) == 0:
                continue
            code2 = ' '.join(c)
        lev_score = lev.distance(code1, code2)
        currentWeight = weight[entry]
        # print ("comparing %s with %s for %s (%0.2f: weight %0.2f)" % (code1, code2, entry, lev_score, currentWeight))
        subtotal = lev_score * currentWeight
        total += subtotal

    return total

In [ ]:
starting_id = 0
sql_1 = f'SELECT * FROM district_discrepencies dd WHERE boundaries_adm0_name IS NULL AND id > {starting_id} ORDER BY id ASC LIMIT 2000'
df_1 = pd.read_sql_query(sql_1, con)
df_1

In [ ]:
sql_2 = 'SELECT * FROM admin2boundaries a2 WHERE '
for g in df_1.groupby(['idsr_adm0_name', 'idsr_adm1_name']).groups:
    # sql_2 += '(lower(adm0_name) = lower(\'' + g[0].replace("'", "''") + '\') AND lower(adm1_name) = lower(\'' + g[1].replace("'", "''") + '\')) OR '
    sql_2 += 'lower(adm0_name) = lower(\'' + g[0].replace("'", "''") + '\') OR '
sql_2 = sql_2[:-4]
df_2 = pd.read_sql_query(sql_2, con)
df_2

In [ ]:
df_2['adm0_name_lower'] = df_2['adm0_name'].str.lower()
df_2['adm1_name_lower'] = df_2['adm1_name'].str.lower()
df_2['adm2_name_lower'] = df_2['adm2_name'].str.lower()

In [ ]:
# this function converts non-latin characters to their closest latin character if possible.
# For example, 'bahaãƒâ¯ alifa' to 'bahaaa-alifa'.
import unicodedata

def get_name_slug(name):
    formatted_name = name.lower().replace(' ', '-').replace('ı','i')
    slug =  unicodedata.normalize('NFD', formatted_name).encode('ascii', 'ignore')

    return slug.decode('utf-8')

In [ ]:
number_of_sounds = 8
sim_measure = 0.31
matches = {}
for idx, row in df_1.iterrows():
    idsr_adm2_name = row['idsr_adm2_name'].lower()
    idsr_adm1_name = row['idsr_adm1_name'].lower()
    if not all(x.isalpha() or x.isspace() for x in idsr_adm2_name) and not all(x.isalpha() or x.isspace() for x in idsr_adm1_name):
        # print(idsr_adm2name)
        continue
    
    df_3 = df_2[df_2['adm0_name_lower'] == row['idsr_adm0_name'].lower()]
    for idx2, row2 in df_3.iterrows():
        boundaries_adm2_name = row2['adm2_name_lower']
        boundaries_adm1_name = row2['adm1_name_lower']
        if not all(x.isalpha() or x.isspace() for x in boundaries_adm2_name) and not all(x.isalpha() or x.isspace() for x in boundaries_adm1_name):
            continue

        try:
            sim_score_2 = compare_words_similarity(idsr_adm2_name, boundaries_adm2_name)
        except IndexError as ex:
            sim_score_2 = compare_words_similarity(get_name_slug(idsr_adm2_name), get_name_slug(boundaries_adm2_name))
        try:
            idsr_sound_2 = ' '.join([soundex(w, number_of_sounds) for w in idsr_adm2_name.split(' ')])
        except IndexError as ex:
            idsr_sound_2 = ' '.join([soundex(w, number_of_sounds) for w in get_name_slug(idsr_adm2_name).split(' ')])
        try:
            boundaries_sound_2 = ' '.join([soundex(w, number_of_sounds) for w in boundaries_adm2_name.split(' ')])
        except IndexError as ex:
            boundaries_sound_2 = ' '.join([soundex(w, number_of_sounds) for w in get_name_slug(boundaries_adm2_name).split(' ')])
            
        if sim_score_2 < sim_measure or (idsr_sound_2 == boundaries_sound_2 and sim_score_2 < 1):
            try:
                sim_score_1 = compare_words_similarity(idsr_adm1_name, boundaries_adm1_name)
            except IndexError as ex:
                sim_score_1 = compare_words_similarity(get_name_slug(idsr_adm1_name), get_name_slug(boundaries_adm1_name))
            try:
                idsr_sound_1 = ' '.join([soundex(w, number_of_sounds) for w in idsr_adm1_name.split(' ')])
            except IndexError as ex:
                idsr_sound_1 = ' '.join([soundex(w, number_of_sounds) for w in get_name_slug(idsr_adm1_name).split(' ')])
            try:
                boundaries_sound_1 = ' '.join([soundex(w, number_of_sounds) for w in boundaries_adm1_name.split(' ')])
            except IndexError as ex:
                boundaries_sound_1 = ' '.join([soundex(w, number_of_sounds) for w in get_name_slug(boundaries_adm1_name).split(' ')])
                
            if sim_score_1 < sim_measure or (idsr_sound_1 == boundaries_sound_1 and sim_score_1 < 1):
                add_match = (row['idsr_adm0_name'], row['idsr_adm1_name'], row['idsr_adm2_name'], row2['adm0_name'], row2['adm1_name'], row2['adm2_name'], 1)
                match_key = f"{row['idsr_adm0_name']} - {row['idsr_adm1_name']} - {row['idsr_adm2_name']}"
                if match_key in matches:
                    # there is already a match for it
                    if sim_score_1 < matches[match_key]['sim_score']:
                        # this means the second match is a better match so add it.
                        matches[match_key] = {
                            'sim_score': sim_score_1,
                            'match': add_match
                        }
                else:     
                    matches[match_key] = {
                        'sim_score': sim_score_1,
                        'match': add_match
                    }

In [ ]:
compare_words_similarity(get_name_slug('bahaãƒâ¯ alifa'), get_name_slug('bahaãƒâ¯ bitkine'))
get_name_slug('bahaãƒâ¯ alifa')

In [ ]:
import csv

with open('district_matches.csv', 'w', newline ='', encoding="utf-8") as f:
    write = csv.writer(f)
    write.writerow(['idsr_adm0name', 'idsr_adm1name', 'idsr_adm2name', 'adm0_name', 'adm1_name', 'adm2_name', 'match'])
    write.writerows(matches)

In [ ]:
df_4 = pd.read_csv('district_matches_done.csv', encoding='utf8')
for idx, row in df_4[df_4['match'] == 1].iterrows():
    idsr_adm0_name = row[0].replace("'", "''")
    idsr_adm1_name = row[1].replace("'", "''")
    idsr_adm2_name = row[2].replace("'", "''")
    boundaries_adm0_name = row[3].replace("'", "''")
    boundaries_adm1_name = row[4].replace("'", "''")
    boundaries_adm2_name = row[5].replace("'", "''")
    sql_3 = f"""
        UPDATE district_discrepencies dd SET 
        boundaries_adm0_name='{boundaries_adm0_name}',
        boundaries_adm1_name='{boundaries_adm1_name}',
        boundaries_adm2_name='{boundaries_adm2_name}' 
        WHERE 
        idsr_adm0_name='{idsr_adm0_name}' AND idsr_adm1_name='{idsr_adm1_name}' AND idsr_adm2_name='{idsr_adm2_name}';
    """
    # print(sql_3)
    con.execute(sql_3)

# Download and process supplemental data

Most of the datasets below follow a similar processing workflow with only small variations:
1. Write a txt file that contains all URLS to download the data.
2. Run wget with the txt file to download files.
3. Run the ZonalStatisticsAsTable ArcGIS Pro tool to get summary values by district. The statistics is either the MEAN or SUM of cell values depending on what makes snese for the data.
4. Import the summary statistics into the DB and query to find any missing districts that were missed because they are too small or on the edge of the data and no cell is completely within it.
5. Find values for the missing districts by getting the value of the nearest cell.
6. Import the missing values into the same DB table as above.

# Download precititation data

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from dateutil import rrule
from datetime import datetime, timedelta

start_date = datetime.strptime('2019-01-01', "%Y-%m-%d") - timedelta(days=7)
end_date = datetime.strptime('2022-12-31', "%Y-%m-%d")
download_path = '/path-to-data/precipitation/'
user = 'scottpez'
passwd = '5c4Nad%CyX'
basic = HTTPBasicAuth(user, passwd)
urls = []

for dt in rrule.rrule(rrule.DAILY, dtstart=start_date, until=end_date):
    y = datetime.strftime(dt, '%Y')
    m = datetime.strftime(dt, '%m')
    d = datetime.strftime(dt, '%d')
    file_name = f'3B-DAY-E.MS.MRG.3IMERG.{y}{m}{d}-S000000-E235959.V06.nc4'
    url = f'https://gpm1.gesdisc.eosdis.nasa.gov/data/GPM_L3/GPM_3IMERGDE.06/{y}/{m}/{file_name}'
    urls.append(url)

with open(download_path + 'urls.txt', 'w') as f:
    for url in urls:
        f.write(url + '\n')

Run the command below in a comand prompt to download the files from the list of URLs in urls.txt produced above.
```
wget --content-disposition --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies --content-disposition --user=scottpez --password=5c4Nad%CyX -i urls.txt
```

# Process precipitation data

In [ ]:
from datetime import datetime, timedelta
from dateutil import rrule
from arcpy.sa import Raster

arcpy.env.overwriteOutput = True
start_date = datetime.strptime('2019-01-01', "%Y-%m-%d") - timedelta(days=7)
end_date = datetime.strptime('2022-11-30', "%Y-%m-%d")
arcpy.env.workspace = '/path-to-data/precipitation/'

for dt in rrule.rrule(rrule.DAILY, dtstart=start_date, until=end_date):
    y = datetime.strftime(dt, '%Y')
    m = datetime.strftime(dt, '%m')
    d = datetime.strftime(dt, '%d')
    file_name = f'3B-DAY-E.MS.MRG.3IMERG.{y}{m}{d}-S000000-E235959.V06.nc4'
    tif_name = file_name.replace('.nc4', '.tif')
    if arcpy.Exists(file_name) and not arcpy.Exists(tif_name):
        arcpy.md.MakeNetCDFRasterLayer(file_name, "HQprecipitation", "lon", "lat", "precipitation", "time", None, "BY_VALUE", "CENTER")
        r = Raster('precipitation')
        r.save(tif_name)

### Extract the HQprecipitation (high-quality precipitation) channel from the nc4 file and save it as a tif.

In [ ]:
import os
import xarray as xr 
import rioxarray as rio
from dask.utils import SerializableLock

lock = SerializableLock()

path = '/path-to-data/precipitation/'
for f in os.listdir(path):
    if f.endswith('.nc4') and not os.path.exists(path + os.sep + f.replace('.nc4', '.tif')):
        print(f'doing {f}')
        nc_file = xr.open_dataset(path + os.sep + f)
        p = nc_file["HQprecipitation"]
        p = p.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
        p.rio.crs
        p.rio.write_crs("epsg:4326", inplace=True)
        p = p.transpose('time', 'lat', 'lon')
        p = p.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
        p.rio.to_raster(path + os.sep + f.replace('.nc4', '.tif'), lock=lock)

#### Using arcpy, find the statistics for precipitation within every administrative district and save as a csv file.

In [ ]:
from datetime import timedelta
import arcpy
from arcpy.ia import *
import pandas as pd


arcpy.env.overwriteOutput = True
idsr_weeks_file = '/path-to-data/20230625_idsr_weeks.csv' # this file simply contains a list exported from IDSR with each week on one line.
df_5 = pd.read_csv(idsr_weeks_file, encoding='utf8')

def group_rasters(path, dir_name, stat):
    
    arcpy.env.workspace = path + dir_name + '/'
    
    groups = {}
    for i, row in df_5.iterrows():
        d2 = pd.to_datetime(row['epidemic_week'])
        d1 = d2 - timedelta(days=7)
        grp = []
        while d1 < d2:
            d_format = d1.strftime('%Y%m%d')
            if dir_name == 'precipitation':
                raster_name = f'3B-DAY-E.MS.MRG.3IMERG.{d_format}-S000000-E235959.V06.tif'
            elif dir_name == 'temperature':
                raster_name = f'GLDAS_NOAH025_3H.A{d_format}.1500.021.tif'
            grp.append(raster_name)
            d1 = d1 + timedelta(days=1)
        groups[d2.strftime('%Y%m%d')] = grp
    
    for k in groups.keys():
        if dir_name == 'precipitation':
            raster_name = f'3B-DAY-E.MS.MRG.3IMERG.{k}-S000000-E235959.V06_week.tif'
        elif dir_name == 'temperature':
            raster_name = f'GLDAS_NOAH025_3H.A{k}.1500.021_week.tif'
        if not arcpy.Exists(raster_name):
            print(f'doing {k}')
            rc = RasterCollection(groups[k])
            if stat == 'SUM':
                range_raster = Sum(rc)
            elif stat == 'MEAN':
                range_raster = Mean(rc)
            range_raster.save(raster_name)

In [ ]:
group_rasters('/path-to-data/', 'precipitation', 'MEAN')

In [ ]:
import os
import arcpy


arcpy.env.overwriteOutput = True


def find_zonal_statistics_for_districts(arc_workspace, stat):
    arcpy.env.workspace = arc_workspace

    admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'
    gdb_path = '/path-to-geodatabase/WHO.gdb'  # I used a geodatabase for this because when I used TIF files it seemed to crash often.

    rasters = arcpy.ListRasters("*", "TIF")
    for raster in rasters:
        out_csv = raster.replace('.', '_').replace('_tif', 'test.csv')
        if '_week' in raster and not os.path.exists(f'{arcpy.env.workspace}/{out_csv}'):
            print(f'doing {raster}')
            out_table = 'temp'
            with arcpy.EnvManager(cellSize=0.125):
                arcpy.ia.ZonalStatisticsAsTable(
                    in_zone_data=admin_shp,
                    zone_field="MY_ID",
                    in_value_raster=raster,
                    out_table=f'{gdb_path}/{out_table}',
                    ignore_nodata="DATA",
                    statistics_type=stat,
                    process_as_multidimensional="CURRENT_SLICE",
                    percentile_values=[90],
                    percentile_interpolation_type="AUTO_DETECT",
                    circular_calculation="ARITHMETIC",
                    circular_wrap_value=360
                )
            arcpy.conversion.ExportTable(f'{gdb_path}/{out_table}', f'{arcpy.env.workspace}/{out_csv}')

            arcpy.management.Delete(f'{gdb_path}/{out_table}')
            return

In [ ]:
find_zonal_statistics_for_districts('/path-to-data/precipitation/', 'MEAN')

Note, that the zonal statistics tool above will miss polygons if those polygons are very small and the boundary does not overlap the center of any raster grid cell. To solve this issue, I will perform the steps below after finding the adm2_code for the polygons that were missed. Basically, it converts the raster to points and finds the nearest point to the missing polygons.

In [ ]:
import os
import csv
import arcpy


def find_zonal_statistics_for_districts_missing(arc_workspace, stat, missing_file_name):
    arcpy.env.workspace = arc_workspace
    arcpy.env.overwriteOutput = True

    admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'
    gdb_path = '/path-to-geodatabase/WHO.gdb'
    temp_points_layer = 'temp_points'

    myids_not_matched = []
    with open(f'{arcpy.env.workspace}/{missing_file_name}', newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        headers = next(csvreader, None)
        for row in csvreader:
            myids_not_matched.append(row[0])

    tmp_clause = ','.join(myids_not_matched)
    tmp_clause = tmp_clause.replace('"', '\'')
    where_clause = f"Admin2_Afro_Master_final.MY_ID IN ({tmp_clause})"
    sel_features = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=admin_shp,
        selection_type="NEW_SELECTION",
        where_clause=where_clause,
        invert_where_clause=None
    )
    ext = arcpy.Describe(sel_features).extent
    buf = 0.1
    ext = arcpy.Extent(ext.XMin-buf, ext.YMin-buf, ext.XMax+buf, ext.XMax+buf)
    with arcpy.EnvManager(extent=ext):
        rasters = arcpy.ListRasters("*", "TIF")
        for raster in rasters:
            out_csv = raster.replace('.', '_').replace('_tif', '_2.csv')
            if '_week' in raster and not os.path.exists(f'{arcpy.env.workspace}/{out_csv}'):
                print(f'doing {raster}')
                arcpy.conversion.RasterToPoint(
                    in_raster=raster,
                    out_point_features=f"{gdb_path}/{temp_points_layer}",
                    raster_field="Value"
                )
                result = arcpy.management.AddSpatialJoin(
                    target_features=sel_features,
                    join_features=f"{gdb_path}/{temp_points_layer}",
                    join_operation="JOIN_ONE_TO_ONE",
                    join_type="KEEP_ALL",
                    field_mapping=f'pointid "pointid" true true false 4 Long 0 0,First,#,{gdb_path}/{temp_points_layer},pointid,-1,-1;grid_code "grid_code" true true false 4 Float 0 0,First,#,{gdb_path}/{temp_points_layer},grid_code,-1,-1',
                    match_option="CLOSEST_GEODESIC",
                    search_radius=0.1,
                    distance_field_name=""
                )
                fields = arcpy.ListFields(result)
                find_fields = []
                for field in fields:
                    if 'MY_ID' in field.name or 'grid_code' in field.name:
                        find_fields.append(field.name)

                result_rows = []
                with arcpy.da.SearchCursor(result, find_fields) as cursor:
                    for row in cursor:
                        result_rows.append([row[0], 1, row[1]])

                with open(f'{arcpy.env.workspace}/{out_csv}', 'w', newline='') as outcsv:
                    writer = csv.writer(outcsv)
                    writer.writerow(["MY_ID", "COUNT", stat])
                    for r in result_rows:
                        writer.writerow(r)

                arcpy.management.Delete(f"{gdb_path}/{temp_points_layer}")
        arcpy.management.Delete(f"{gdb_path}/sel_features_temp")

#### Read in all precipitation CSV files and aggregate the daily precipitation to the epidemic weeks and divide by the  to find the average temperature in the district for each week.

In [ ]:
import os
import csv
from datetime import datetime
import pandas as pd


def group_statistics_by_district_time(dir_name, stat, step=1, clas=None):
    path = f'/path-to-data/{dir_name}/'
    dfs = []
    for f in os.listdir(path):
        if (dir_name == 'temperature' and '_week' in f and f.endswith('.csv') and f.startswith('GLDAS')) \
        or (dir_name == 'precipitation' and '_week' in f and f.endswith('.csv') and f.startswith('3B-DAY')) \
        or (dir_name == 'landcover' and f.endswith(f'{clas}_week.csv')):
            if (step == 1 and '_2.csv' not in f) or step == 2:
                if '_2.csv' in f:
                    df = pd.read_csv(path + f, usecols = ['MY_ID', 'COUNT', 'SUM'])
                    df.rename(columns = {'SUM':'MEAN'}, inplace = True)
                else:
                    df = pd.read_csv(path + f, usecols = ['MY_ID', 'COUNT', stat])
                # df = pd.read_csv(path + f, usecols = ['MY_ID', 'COUNT', stat])
                if dir_name == 'temperature':
                    date_start_idx = 18
                elif dir_name == 'precipitation':
                    date_start_idx = 23
                dd = datetime.strptime(f[date_start_idx:date_start_idx+8], '%Y%m%d')
                df['date'] = dd
                dfs.append(df)
    df = pd.concat(dfs, axis=0)
    
    with open(f'{path}{dir_name}_full_{step}.csv', 'w', newline='') as csvfile:
        fieldnames = ['my_id', 'epidemic_week', f'val_{dir_name}']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for idx, row in df.iterrows():
            val = row[stat]
            if dir_name == 'temperature':
                val = val - 273.15  # convert kelvin temperature to celsius
            writer.writerow({'my_id': row['MY_ID'], 'epidemic_week': row['date'], f'val_{dir_name}': val})

Note, that the zonal statsitics tool above will miss polygons if those polygons are very small and the boundary does not overlap the center of any raster grid cell. To solve this issue, I will perform the steps below after finding the adm2_code for the polygons that were missed. Basically, it converts the raster to points and finds the nearest point to the missing polygons.

In [ ]:
group_statistics_by_district_time('precipitation', 'MEAN', 1)

1. Above, I grouped the statistics for precipitation by adm2 and week.
2. Next, import this CSV file into the database and run the query to see which districts were not matched to the precipitation grid either because they are too small or right on an edge.
3. Find all districts not matched and save their adm2_code's to a CSV file to be read in below.
4. Once all districts are matched, then reimport the precipitation CSV file.

In [ ]:
find_zonal_statistics_for_districts_missing('/path-to-data/precipitation/', 'MEAN', '20230626_districtsWithNoPrecipitationMatch.csv')

In [ ]:
group_statistics_by_district_time('precipitation', 'MEAN', 2)

### Simplify precipitation raster and clip

Note, I was previously simplifying the rasters because I was running the zonal statistics in PostGIS. It was much faster in ArcGIS Pro so I didn't have to do this anymore. So, I am not using the code in the cell below.

# Temperature data

## Download the temperature data

Below it will create a txt file that contains the URLs for all iamges to download

In [ ]:
import requests
from requests.auth import HTTPBasicAuth
from dateutil import rrule
from datetime import datetime, timedelta

start_date = datetime.strptime('2019-01-01', "%Y-%m-%d") - timedelta(days=7)
end_date = datetime.strptime('2022-02-17', "%Y-%m-%d")
download_path = '/path-to-data/temperature/'
user = 'scottpez'
passwd = '5c4Nad%CyX'
basic = HTTPBasicAuth(user, passwd)
urls = []

for dt in rrule.rrule(rrule.DAILY, dtstart=start_date, until=end_date):
    y = datetime.strftime(dt, '%Y')
    m = datetime.strftime(dt, '%m')
    d = datetime.strftime(dt, '%d')
    w = datetime.strftime(dt, '%j')
    h = '15'
    mi = '00'
    
    file_name = f'GLDAS_NOAH025_3H.A{y}{m}{d}.{h}{mi}.021.nc4'
    url = f'https://data.gesdisc.earthdata.nasa.gov/data/GLDAS/GLDAS_NOAH025_3H.2.1/{y}/{w}/{file_name}'
    urls.append(url)

with open(download_path + 'urls.txt', 'w') as f:
    for url in urls:
        f.write(f'{url}\n')

#### Now, run the wget command below to download all image files listed in the text file created above.

Run the command below in a command line.

wget --content-disposition --load-cookies ~/.urs_cookies --save-cookies ~/.urs_cookies --keep-session-cookies --content-disposition --user=scottpez --password=5c4Nad%CyX -i urls.txt

The above command will save the file name with a lengthy extension that adds the parameters. So, next, create a bash shell script in the same folder, save it, and run it.

```
#!/bin/sh

for f in *
  do
  name=$(echo $f| cut -c 1-39)
  command=$(mv $f $name)
  $command
done
```

#### From the NC4 file, extract the layer for temperature to its own TIF file.

In [ ]:
import os
import xarray as xr 
import rioxarray as rio
from dask.utils import SerializableLock

lock = SerializableLock()

path = '/path-to-data/temperature/'
for f in os.listdir(path):
    tif_path = path + f.replace('.nc4', '.tif')
    if f.endswith('.nc4') and not os.path.exists(tif_path):
        print(f)
        nc_file = xr.open_dataset(path + os.sep + f)
        p = nc_file["Tair_f_inst"]
        p = p.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
        p.rio.crs
        p.rio.write_crs("epsg:4326", inplace=True)
        p = p.transpose('time', 'lat', 'lon')
        p = p.rio.set_spatial_dims(x_dim='lon', y_dim='lat')
        p.rio.to_raster(path + os.sep + f.replace('.nc4', '.tif'), lock=lock)

In [ ]:
group_rasters('/path-to-data/', 'temperature', 'MEAN')

#### Using arcpy, find the statistics for temperature within every administrative district and save as a csv file.

In [ ]:
find_zonal_statistics_for_districts('/path-to-data/temperature/', 'MEAN')

Note, that the zonal statsitics tool above will miss polygons if those polygons are very small and the boundary does not overlap the center of any raster grid cell. To solve this issue, I will perform the steps below after finding the adm2_code for the polygons that were missed. Basically, it converts the raster to points and finds the nearest point to the missing polygons.

In [ ]:
group_statistics_by_district_time('temperature', 'MEAN', 1)

1. Above, I grouped the statistics for precipitation by adm2 and week.
2. Next, import this CSV file into the database and run the query to see which districts were not matched to the precipitation grid either because they are too small or right on an edge.
3. Find all districts not matched and save their adm2_code's to a CSV file to be read in below.
4. Once all districts are matched, then reimport the precipitation CSV file.

In [ ]:
find_zonal_statistics_for_districts_missing('/path-to-data/temperature/', 'MEAN', '20230627_districtsWithNoTemperatureMatch.csv')

#### Read in all temperature CSV files and aggregate the daily temperature to the epidemic weeks to find the average temperature in the district for each week.

In [ ]:
group_statistics_by_district_time('temperature', 'MEAN', 2)

# Download landcover data

Hosted by Microsoft.

https://planetarycomputer.microsoft.com/dataset/io-lulc-9-class

Produced by Impact Observatory using deep learning model on Copernicus Sentinel 2 imagery.

https://www.impactobservatory.com/global_maps/.

In [ ]:
import os
import requests
import shutil

download_path = '/path-to-data/landcover/'
rows = ['H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S']
cols = [26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41]
years = [2018, 2020, 2021, 2022]

for year in years:
    if not os.path.exists(download_path + str(year)):
        os.mkdir(download_path + str(year))
    for col in cols:
        for row in rows:
            file_name = f'{col}{row}_{year}0101-{year+1}0101.tif'
            
            if not os.path.exists(download_path + f'{year}/{file_name}'):
                print('Downloading: ',file_name)
                url = f'https://lulctimeseries.blob.core.windows.net/lulctimeseriespublic/lc{year}/{col}{row}_{year}0101-{year+1}0101.tif'
                r = requests.get(url, stream=True)

                # Check if the image was retrieved successfully
                if r.status_code == 200:
                    # Set decode_content value to True, otherwise the downloaded image file's size will be zero.
                    r.raw.decode_content = True

                    # Open a local file with wb ( write binary ) permission.
                    with open(download_path + f'{year}/{file_name}', 'wb') as f:
                        shutil.copyfileobj(r.raw, f)

# Process landcover

### Create mosaic for landcover, clip it to the boundary of Africa, and save it as a tif

In [ ]:
import os
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False
gdb_path = '/path-to-geodatabase/WHO.gdb'

years = [2018, 2019, 2020, 2021, 2022]

for year in years:
    print(f'doing {year}')
    arcpy.env.workspace = '/path-to-data/landcover' + os.sep + str(year) + os.sep
    raster_names = []
    rasters = arcpy.ListRasters("*", "TIF")
    for raster in rasters:
        new_raster = raster.replace('.tif', '_4326.tif')
        if not '4326' in raster and not arcpy.Exists(new_raster):
            print(f'doing {raster}')
            arcpy.management.ProjectRaster(
                in_raster=raster,
                out_raster=new_raster,
                out_coor_system='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'
            )

In [ ]:
import os
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False
gdb_path = '/path-to-data/WHO.gdb'

years = [2018, 2019, 2020, 2021, 2022]

for year in years:
    print(f'doing {year}')
    arcpy.env.workspace = '/path-to-data/landcover' + os.sep + str(year) + os.sep
    raster_names = []
    rasters = arcpy.ListRasters("*", "TIF")
    for raster in rasters:
        if '4326' in raster:
            raster_names.append(raster)
    all_rasters = ';'.join(raster_names)
    arcpy.management.CreateRasterDataset(gdb_path, f"lc_{year}", "0.00736", "8_BIT_UNSIGNED", None)
    arcpy.management.Mosaic(
        inputs=all_rasters,
        target=gdb_path + os.sep + f"lc_{year}",
        mosaic_type="LAST",
        colormap="FIRST",
        background_value=None,
        nodata_value=None,
        onebit_to_eightbit="NONE",
        mosaicking_tolerance=0,
        MatchingMethod="NONE"
    )

#### Separate the values for different land covers and do zonal statistics

In [ ]:
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

years = [2018, 2019, 2020, 2021, 2022]
clas = {
    'trees': 2, # this is the class value for trees
    'crops': 5,
    'builtup': 7,
    'bareground': 8,
    'rangeland': 11
}
out_table = 'temp'
gdb_path = '/path-to-geodatabase/WHO.gdb'
arcpy.env.workspace = gdb_path
admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'

for year in years:
    raster = f"lc_{year}"
    for k, v in clas.items():
        print(f'doing {raster}_{k}')
        out_csv = f'{raster}_{k}.csv'
        sub_ras = arcpy.ia.Con(
            in_conditional_raster=raster,
            in_true_raster_or_constant=1,
            in_false_raster_or_constant=0,
            where_clause=f"VALUE = {v}"
        ) # if it is the class then set it equal to 1, if not 0.
        arcpy.ia.ZonalStatisticsAsTable(
            in_zone_data=admin_shp,
            zone_field="MY_ID",
            in_value_raster=sub_ras,
            out_table=f'{gdb_path}/{out_table}',
            ignore_nodata="DATA",
            statistics_type='SUM'
        )
        arcpy.conversion.ExportTable(f'{gdb_path}/{out_table}', f'/path-to-data/landcover/{year}/{out_csv}')
        arcpy.management.Delete(f'{gdb_path}/{out_table}')

In [ ]:
import arcpy


arcpy.env.workspace = '/path-to-geodatabase/WHO.gdb'
years = [2018, 2019, 2020, 2021, 2022]

for year in years:
    raster = f"lc_{year}"
    arcpy.management.CalculateStatistics(raster)

In [ ]:
import csv
import os
import arcpy


years = [2018, 2019, 2020, 2021, 2022]
clas = {
    'trees': 2, # this is the class value for trees
    'crops': 5,
    'builtup': 7,
    'bareground': 8,
    'rangeland': 11
}
arcpy.env.workspace = '/path-to-geodatabase/WHO.gdb'
arcpy.env.overwriteOutput = True
stat = 'SUM'


for year in years:
    raster = f"lc_{year}"
    admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'
    gdb_path = '/path-to-geodatabase/WHO.gdb'
    temp_points_layer = 'temp_points'
    missing_file_name = '/path-to-data/20230630_districtsWithNoLandcoverMatch.csv'

    myids_not_matched = []
    with open(missing_file_name, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        headers = next(csvreader, None)
        for row in csvreader:
            myids_not_matched.append(row[0])

    tmp_clause = ','.join(myids_not_matched)
    tmp_clause = tmp_clause.replace('"', '\'')
    where_clause = f"Admin2_Afro_Master_final.MY_ID IN ({tmp_clause})"
    sel_features = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=admin_shp,
        selection_type="NEW_SELECTION",
        where_clause=where_clause,
        invert_where_clause=None
    )
    ext = arcpy.Describe(sel_features).extent
    buf = 0.1
    ext = arcpy.Extent(ext.XMin-buf, ext.YMin-buf, ext.XMax+buf, ext.XMax+buf)
    with arcpy.EnvManager(extent=ext):
        out_csv = f'{raster}_2.csv'
        if not os.path.exists(f'/path-to-data/landcover/{year}/{out_csv}'):
            print(f'doing {raster}')
            arcpy.conversion.RasterToPoint(
                in_raster=raster,
                out_point_features=f"{gdb_path}/{temp_points_layer}",
                raster_field="Value"
            )
            result = arcpy.management.AddSpatialJoin(
                target_features=sel_features,
                join_features=f"{gdb_path}/{temp_points_layer}",
                join_operation="JOIN_ONE_TO_ONE",
                join_type="KEEP_ALL",
                field_mapping=f'pointid "pointid" true true false 4 Long 0 0,First,#,{gdb_path}/{temp_points_layer},pointid,-1,-1;grid_code "grid_code" true true false 4 Float 0 0,First,#,{gdb_path}/{temp_points_layer},grid_code,-1,-1',
                match_option="CLOSEST_GEODESIC",
                search_radius=0.1,
                distance_field_name=""
            )
            fields = arcpy.ListFields(result)
            find_fields = []
            for field in fields:
                if 'MY_ID' in field.name or 'grid_code' in field.name:
                    find_fields.append(field.name)

            result_rows = []
            with arcpy.da.SearchCursor(result, find_fields) as cursor:
                for row in cursor:
                    result_rows.append([row[0], 1, row[1]])

            with open(f'/path-to-data/landcover/{year}/{out_csv}', 'w', newline='') as outcsv:
                writer = csv.writer(outcsv)
                writer.writerow(["MY_ID", "COUNT", "CLAS"])
                for r in result_rows:
                    writer.writerow(r)

            arcpy.management.Delete(f"{gdb_path}/{temp_points_layer}")
    arcpy.management.Delete(f"{gdb_path}/sel_features_temp")

In [ ]:
import csv
import pandas as pd

years = [2018, 2019, 2020, 2021, 2022]
clas = {
    'trees': 2, # this is the class value for trees
    'crops': 5,
    'builtup': 7,
    'bareground': 8,
    'rangeland': 11
}


def group_landcover(dir_name, stat, step=1, clas=None):
    path = f'/path-to-data/{dir_name}/'
    dfs = []
    for y in years:
        f = f'{path}{y}/lc_{y}_2.csv'
        df_1 = pd.read_csv(f, usecols = ['MY_ID', 'COUNT', 'CLAS'])
        lookup = {}
        for idx, row in df_1.iterrows():
            my_id = row[0]
            my_clas = row[2]
            for k, v in clas.items():
                if k not in lookup:
                    lookup[k] = []
                if v == my_clas:
                    lookup[k].append([my_id, 1])
                else:
                    lookup[k].append([my_id, 0])
                
        for k in clas.keys():
            f = f'{path}{y}/lc_{y}_{k}.csv'
            df = pd.read_csv(f, usecols = ['MY_ID', 'COUNT', stat])
            new_data = []
            for r in lookup[k]:
                new_data.append([r[0], 1, r[1]])
            new_df = pd.DataFrame(new_data, columns=['MY_ID', 'COUNT', stat])
            df = pd.concat([df, new_df])
            df['year'] = y
            df['clas'] = k
            dfs.append(df)
    df = pd.concat(dfs, axis=0)
    df['PERCENT_COV'] = df[stat] / df['COUNT']
    
    with open(f'{path}{dir_name}_full_{step}.csv', 'w', newline='') as csvfile:
        fieldnames = ['my_id', 'year', 'clas', 'val', 'percent_cov']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)

        writer.writeheader()
        for i, row in df.iterrows():
            writer.writerow({'my_id': row['MY_ID'], 'year': row['year'], 'clas': row['clas'], 'val': row[stat], 'percent_cov': row['PERCENT_COV']})

In [ ]:
group_landcover('landcover', 'SUM', 2, clas)

# Create data that shows the population and population near water bodies

In [ ]:
import arcpy
from arcpy.sa import *

admin_shp = '/path-to-data/Admin2_Afro_Master_subset1.shp'
gdb_path = '/path-to-geodatabase/WHO.gdb'
waterbodies_buffer = '/path-to-data/osm_waterbodies_3km.shp'
new_waterbodies_buffer = waterbodies_buffer.split('\\')[-1].replace('.shp', '.tif')
arcpy.conversion.PolygonToRaster(
    in_features=waterbodies_buffer,
    value_field="FID",
    out_rasterdataset=gdb_path + os.sep + new_waterbodies_buffer,
    cell_assignment="CELL_CENTER",
    priority_field="NONE",
    cellsize="/path-to-data/population/ppp_2020_1km_Aggregated.tif",
    build_rat="BUILD"
)

In [ ]:
import arcpy
from arcpy.sa import *

arcpy.env.overwriteOutput = True

new_waterbodies_buffer = '/path-to-geodatabase/WHO.gdb/osm_waterbodies_3km'
years = [2017, 2018, 2019, 2020]
for y in years:
    print(f'doing {y}')
    population_raster = f'/path-to-data/population/ppp_{y}_1km_Aggregated.tif'
    raster = Raster(new_waterbodies_buffer)
    con_temp = Con(in_conditional_raster=raster, in_true_raster_or_constant=population_raster, in_false_raster_or_constant=0, where_clause="VALUE > 0")
    con_raster = Con(
        in_conditional_raster=con_temp,
        in_true_raster_or_constant=0,
        in_false_raster_or_constant=con_temp,
        where_clause="VALUE IS NULL"
    )
    con_raster.save(population_raster.replace('.tif', '_water.tif'))

In [ ]:
import arcpy

arcpy.env.overwriteOutput = True
arcpy.env.addOutputsToMap = False

years = [2017, 2018, 2019, 2020]
out_table = 'temp'
gdb_path = '/path-to-geodatabase/WHO.gdb'
arcpy.env.workspace = gdb_path
admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'

for y in years:
    raster = f'/path-to-data/population/ppp_{y}_1km_Aggregated_water.tif'
    print(f'doing {raster}')
    out_csv = raster.replace('.tif', '.csv')
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=admin_shp,
        zone_field="MY_ID",
        in_value_raster=raster,
        out_table=f'{gdb_path}/{out_table}',
        ignore_nodata="DATA",
        statistics_type='SUM'
    )
    arcpy.conversion.ExportTable(f'{gdb_path}/{out_table}', out_csv)
    arcpy.management.Delete(f'{gdb_path}/{out_table}')
    
    raster = f'/path-to-data/population/ppp_{y}_1km_Aggregated.tif'
    print(f'doing {raster}')
    out_csv = raster.replace('.tif', '.csv')
    arcpy.ia.ZonalStatisticsAsTable(
        in_zone_data=admin_shp,
        zone_field="MY_ID",
        in_value_raster=raster,
        out_table=f'{gdb_path}/{out_table}',
        ignore_nodata="DATA",
        statistics_type='SUM'
    )
    arcpy.conversion.ExportTable(f'{gdb_path}/{out_table}', out_csv)
    arcpy.management.Delete(f'{gdb_path}/{out_table}')

In [ ]:
import csv
import pandas as pd


years = [2017, 2018, 2019, 2020]

for y in years:
    csv_1 = f'/path-to-data/population/ppp_{y}_1km_Aggregated_water.csv'
    csv_2 = f'/path-to-data/population/ppp_{y}_1km_Aggregated.csv'
    
    df_1 = pd.read_csv(csv_1)
    df_1.set_index('MY_ID')
    df_2 = pd.read_csv(csv_2)
    df_2.set_index('MY_ID')
    
    df = df_1.join(df_2, lsuffix='_w', rsuffix='_n', how='outer')
    df['COUNT_w'] = df['COUNT_w'].fillna(0)
    df['SUM_w'] = df['SUM_w'].fillna(0)
    
    with open(f'/path-to-data/population/population_full_{y}.csv', 'w', newline='') as csvfile:
        fieldnames = ['my_id', 'year', 'count_near_water', 'pop_near_water', 'count_total', 'pop_total']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for idx, row in df.iterrows():
            my_id = row['MY_ID_n']
            count_near_water = row['COUNT_w']
            pop_near_water = row['SUM_w']
            count_total = row['COUNT_n']
            pop_total = row['SUM_n']
            writer.writerow({
                'my_id': my_id,
                'year': y,
                'count_near_water': count_near_water,
                'pop_near_water': pop_near_water,
                'count_total': count_total,
                'pop_total': pop_total
            })

In [ ]:
import csv
import os
import arcpy


years = [2017, 2018, 2019, 2020]
arcpy.env.workspace = '/path-to-data/population/'
arcpy.env.overwriteOutput = True
stat = 'SUM'

results = {}
for y in years:
    raster = f'ppp_{y}_1km_Aggregated.tif'
    admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'
    gdb_path = '/path-to-geodatabase/WHO.gdb'
    temp_points_layer = 'temp_points'
    missing_file_name = f'/path-to-data/population/20230723_districtsWithNoPopulationMatch.csv'

    myids_not_matched = []
    with open(missing_file_name, newline='') as csvfile:
        csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
        headers = next(csvreader, None)
        for row in csvreader:
            myids_not_matched.append(f'"{row[0]}"')

    tmp_clause = ','.join(myids_not_matched)
    tmp_clause = tmp_clause.replace('"', '\'')
    where_clause = f"Admin2_Afro_Master_final.MY_ID IN ({tmp_clause})"
    sel_features = arcpy.management.SelectLayerByAttribute(
        in_layer_or_view=admin_shp,
        selection_type="NEW_SELECTION",
        where_clause=where_clause,
        invert_where_clause=None
    )
    ext = arcpy.Describe(sel_features).extent
    buf = 0.1
    ext = arcpy.Extent(ext.XMin-buf, ext.YMin-buf, ext.XMax+buf, ext.XMax+buf)
    with arcpy.EnvManager(extent=ext):
        out_csv = f'population_full_{y}_3.csv'
        if True or not os.path.exists(f'/path-to-data/population/{out_csv}'):
            print(f'doing {raster}')
            # total
            arcpy.conversion.RasterToPoint(
                in_raster=raster,
                out_point_features=f"{gdb_path}/{temp_points_layer}",
                raster_field="Value"
            )
            result = arcpy.management.AddSpatialJoin(
                target_features=sel_features,
                join_features=f"{gdb_path}/{temp_points_layer}",
                join_operation="JOIN_ONE_TO_ONE",
                join_type="KEEP_ALL",
                field_mapping=f'pointid "pointid" true true false 4 Long 0 0,First,#,{gdb_path}/{temp_points_layer},pointid,-1,-1;grid_code "grid_code" true true false 4 Float 0 0,First,#,{gdb_path}/{temp_points_layer},grid_code,-1,-1',
                match_option="CLOSEST_GEODESIC",
                search_radius=0.1,
                distance_field_name=""
            )
            fields = arcpy.ListFields(result)
            find_fields = []
            for field in fields:
                if 'MY_ID' in field.name or 'grid_code' in field.name:
                    find_fields.append(field.name)

            result_rows = []
            with arcpy.da.SearchCursor(result, find_fields) as cursor:
                for row in cursor:
                    result_rows.append([row[0], row[1]])

            with open(f'/path-to-data/landcover/{year}/{out_csv}', 'w', newline='') as outcsv:
                writer = csv.writer(outcsv)
                writer.writerow(["MY_ID", stat])
                for r in result_rows:
                    writer.writerow(r)
                    
            arcpy.management.Delete(f"{gdb_path}/{temp_points_layer}")
    arcpy.management.Delete(f"{gdb_path}/sel_features_temp")

In [ ]:
with open(f'/path-to-data/population/population_full_{y}_3.csv', 'w', newline='') as csvfile:
    fieldnames = ['my_id', 'year', 'count_near_water', 'pop_near_water', 'count_total', 'pop_total']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for k, v in results.items():
        my_id = k.split('_')[0]
        y = k.split('_')[1]
        count_near_water = v['count_near_water']
        pop_near_water = v['pop_near_water']
        count_total = v['count_total']
        pop_total = v['pop_total']
        writer.writerow({
            'my_id': my_id,
            'year': y,
            'count_near_water': count_near_water,
            'pop_near_water': pop_water,
            'count_total': count_total,
            'pop_total': pop_total
        })

# Create data for the relative wealth

In [ ]:
import csv
import os
import arcpy


arcpy.env.workspace = f'/path-to-data/wealth/'
arcpy.env.overwriteOutput = True
stat = 'MEAN'
out_table = 'temp'

raster = f'/path-to-data/wealth/relative_wealth_index.tif'
admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'
gdb_path = '/path-to-geodatabase/WHO.gdb'
print(f'doing {raster}')
out_csv = raster.replace('.tif', '.csv')
arcpy.ia.ZonalStatisticsAsTable(
    in_zone_data=admin_shp,
    zone_field="MY_ID",
    in_value_raster=raster,
    out_table=f'{gdb_path}/{out_table}',
    ignore_nodata="DATA",
    statistics_type='SUM'
)
arcpy.conversion.ExportTable(f'{gdb_path}/{out_table}', out_csv)
arcpy.management.Delete(f'{gdb_path}/{out_table}')

raster = f'/path-to-data/population/ppp_{y}_1km_Aggregated.tif'
print(f'doing {raster}')
out_csv = raster.replace('.tif', '.csv')
arcpy.ia.ZonalStatisticsAsTable(
    in_zone_data=admin_shp,
    zone_field="MY_ID",
    in_value_raster=raster,
    out_table=f'{gdb_path}/{out_table}',
    ignore_nodata="DATA",
    statistics_type='SUM'
)
arcpy.conversion.ExportTable(f'{gdb_path}/{out_table}', out_csv)
arcpy.management.Delete(f'{gdb_path}/{out_table}')

In [ ]:
in_csv = '/path-to-data/wealth/relative_wealth_index_full.csv'
out_csv = '/path-to-data/wealth/relative_wealth_index_full_2.csv'
df = pd.read_csv(in_csv)
new_df = df[['MY_ID', 'MEAN']]
new_df.to_csv(out_csv, index=False)

# Create data for elevation

In [ ]:
import arcpy


# Add all the dems to a mosaic
arcpy.management.AddRastersToMosaicDataset(
    in_mosaic_dataset=r"/path-to-geodatabase/WHO.gdb/elevation",
    raster_type="Raster Dataset",
    input_path="/path-to-data/data\elevation",
    update_cellsize_ranges="UPDATE_CELL_SIZES",
    update_boundary="UPDATE_BOUNDARY",
    update_overviews="NO_OVERVIEWS",
    maximum_pyramid_levels=None,
    maximum_cell_size=0,
    minimum_dimension=1500,
    spatial_reference=None,
    filter="",
    sub_folder="NO_SUBFOLDERS",
    duplicate_items_action="ALLOW_DUPLICATES",
    build_pyramids="NO_PYRAMIDS",
    calculate_statistics="NO_STATISTICS",
    build_thumbnails="NO_THUMBNAILS",
    operation_description="",
    force_spatial_reference="NO_FORCE_SPATIAL_REFERENCE",
    estimate_statistics="NO_STATISTICS",
    aux_inputs=None,
    enable_pixel_cache="NO_PIXEL_CACHE",
    cache_location=r"C:\Users\<windows_username>\AppData\Local\ESRI\rasterproxies\elevation"
)

In [ ]:
import arcpy


arcpy.env.overwriteOutput = True

# Reduce it to a lower resolution
with arcpy.EnvManager(outputCoordinateSystem='GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', parallelProcessingFactor="80%", extent='-25.3605575569999 -30.6778479549999 56.2956848150001 27.290458679 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]'):
    out_raster = arcpy.sa.Aggregate(
        in_raster=r"/path-to-geodatabase/WHO.gdb/elevation",
        cell_factor=100,
        aggregation_type="MEDIAN",
        extent_handling="EXPAND",
        ignore_nodata="DATA"
    )
out_raster.save(r"/path-to-geodatabase/WHO.gdb/elevation_lowres")

In [ ]:
import csv
import os
import arcpy


arcpy.env.workspace = '/path-to-data/elevation/'
arcpy.env.overwriteOutput = True
stat = 'MEDIAN'

raster = '/path-to-geodatabase/WHO.gdb/elevation_lowres'
admin_shp = '/path-to-data/Admin2_Afro_Master_final.shp'
gdb_path = '/path-to-geodatabase/WHO.gdb'
temp_points_layer = 'temp_points'
missing_file_name = '/path-to-data/elevation/20230709_districtsWithNoElevationMatch.csv'

myids_not_matched = []
with open(missing_file_name, newline='') as csvfile:
    csvreader = csv.reader(csvfile, delimiter=' ', quotechar='|')
    headers = next(csvreader, None)
    for row in csvreader:
        myids_not_matched.append(f'"{row[0]}"')

tmp_clause = ','.join(myids_not_matched)
tmp_clause = tmp_clause.replace('"', '\'')
where_clause = f"Admin2_Afro_Master_final.MY_ID IN ({tmp_clause})"
sel_features = arcpy.management.SelectLayerByAttribute(
    in_layer_or_view=admin_shp,
    selection_type="NEW_SELECTION",
    where_clause=where_clause,
    invert_where_clause=None
)
ext = arcpy.Describe(sel_features).extent
buf = 0.1
ext = arcpy.Extent(ext.XMin-buf, ext.YMin-buf, ext.XMax+buf, ext.XMax+buf)
with arcpy.EnvManager(extent=ext):
    out_csv = f'elevation_full_2.csv'
    if True or not os.path.exists(f'/path-to-data/elevation/{out_csv}'):
        print(f'doing {raster}')
        # total
        arcpy.conversion.RasterToPoint(
            in_raster=raster,
            out_point_features=f"{gdb_path}/{temp_points_layer}",
            raster_field="Value"
        )
        result = arcpy.management.AddSpatialJoin(
            target_features=sel_features,
            join_features=f"{gdb_path}/{temp_points_layer}",
            join_operation="JOIN_ONE_TO_ONE",
            join_type="KEEP_ALL",
            field_mapping=f'pointid "pointid" true true false 4 Long 0 0,First,#,{gdb_path}/{temp_points_layer},pointid,-1,-1;grid_code "grid_code" true true false 4 Float 0 0,First,#,{gdb_path}/{temp_points_layer},grid_code,-1,-1',
            match_option="CLOSEST_GEODESIC",
            search_radius=0.1,
            distance_field_name=""
        )
        fields = arcpy.ListFields(result)
        find_fields = []
        for field in fields:
            if 'MY_ID' in field.name or 'grid_code' in field.name:
                find_fields.append(field.name)

        result_rows = []
        with arcpy.da.SearchCursor(result, find_fields) as cursor:
            for row in cursor:
                result_rows.append([row[0], row[1]])

        with open(f'/path-to-data/elevation/{out_csv}', 'w', newline='') as outcsv:
            writer = csv.writer(outcsv)
            writer.writerow(["MY_ID", stat])
            for r in result_rows:
                writer.writerow(r)
                    
        arcpy.management.Delete(f"{gdb_path}/{temp_points_layer}")
arcpy.management.Delete(f"{gdb_path}/sel_features_temp")